In [1]:
!pip -q install catboost iterative-stratification

import gc
import ast
import json
import numpy as np
import pandas as pd

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.ensemble import VotingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.utils.class_weight import compute_sample_weight

from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data preparing

In [2]:
train_df = pd.read_csv('/kaggle/input/vseros-1e/train_frida_emb.csv')
test_df = pd.read_csv('/kaggle/input/vseros-1e/test_frida_emb.csv')
train_df['text'] = train_df['text'].apply(lambda x: x[len('categorize_topic: '):])
test_df['text'] = test_df['text'].apply(lambda x: x[len('categorize_topic: '):])
print(train_df.shape)
print(test_df.shape)
train_df.head()

(53494, 8)
(15046, 5)


,app_name,full_description,shortDescription,labels_str,labels_list,n_labels,text,embedding
0,Матрона Московская.,Хочешь знать что будет? Загляни в будущие. Мат...,Узнать будущее. Предсказания . Магия волшебног...,lifestyle,['lifestyle'],1,Матрона Московская. Узнать будущее. Предсказан...,"[-0.02546943910419941, -0.02646569162607193, -..."
1,"Run and Jump - ""Бесконечный раннер""",Run and Jump это новая увлекательная Аркада - ...,Платформенная Аркада - Раннер с захватывающим ...,action|arcade,"['action', 'arcade']",2,"Run and Jump - ""Бесконечный раннер"" Платформен...","[-0.03978952392935753, -0.004444682039320469, ..."
2,Ghost Maze,"Играя за приведение, собирай необходимые комби...",Игра-головоломка. Попробуй найти выход из лаби...,arcade|puzzle,"['arcade', 'puzzle']",2,Ghost Maze Игра-головоломка. Попробуй найти вы...,"[-0.04193723574280739, -0.0033695693127810955,..."
3,LabTools Mobile: ЛАБОРАТОРНЫЕ ПРИБОРЫ,AR-приложение «LabTools Mobile: Лабораторные п...,AR-приложение которое предназначено для изучен...,education,['education'],1,LabTools Mobile: ЛАБОРАТОРНЫЕ ПРИБОРЫ AR-прило...,"[-0.044376254081726074, -0.028724001720547676,..."
4,Mario Anime Coloring,\nРаскрасьте своих любимых персонажей из mario...,Раскрасьте своих любимых персонажей из mario я...,children|family,"['children', 'family']",2,Mario Anime Coloring Раскрасьте своих любимых ...,"[-0.03006214275956154, -0.011583167128264904, ..."


In [3]:
def safe_json_loads(x):
    return json.loads(x)

train_df['embedding'] = train_df['embedding'].apply(safe_json_loads)
test_df['embedding'] = test_df['embedding'].apply(safe_json_loads)
gc.collect()

0

In [4]:
train_df['labels_list'] = train_df['labels_list'].apply(lambda x: eval(x))
gc.collect()

0

### Tfidf features

In [5]:
import re

russian_stopwords = list(stopwords.words('russian'))

def russian_tokenizer(text):
    tokens = re.findall(r'[a-zA-Zа-яА-ЯёЁ0-9]+', str(text).lower())
    return [token for token in tokens if len(token) > 1]

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(
    max_features=200,
    ngram_range=(1, 2),
    stop_words=russian_stopwords,        # ← русские стоп-слова
    lowercase=True,
    tokenizer=russian_tokenizer,         # ← кастомный токенизатор
    token_pattern=None,                  # ← отключаем встроенный, используем свой
    strip_accents=None,                  # ← не трогаем кириллицу
    min_df=3,
    max_df=0.8
)

X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['text'].fillna(''))
X_test_tfidf = tfidf_vectorizer.transform(test_df['text'].fillna(''))

In [7]:
# Получаем имена фичей из TF-IDF
tfidf_feature_names = [f"tfidf_{i}" for i in range(X_train_tfidf.shape[1])]

# Конвертируем sparse → dense и добавляем в DataFrame
train_tfidf_dense = X_train_tfidf.toarray()
test_tfidf_dense = X_test_tfidf.toarray()

# Создаём DataFrame с TF-IDF признаками
train_tfidf_df = pd.DataFrame(train_tfidf_dense, columns=tfidf_feature_names, index=train_df.index)
test_tfidf_df = pd.DataFrame(test_tfidf_dense, columns=tfidf_feature_names, index=test_df.index)

print(train_df.shape, test_df.shape)
# Конкатенируем с исходными датафреймами
train_df = pd.concat([train_df, train_tfidf_df], axis=1)
test_df = pd.concat([test_df, test_tfidf_df], axis=1)
print(train_df.shape, test_df.shape)

(53494, 8) (15046, 5)
(53494, 208) (15046, 205)


In [8]:
def add_text_feats(df, text_col):
    s = df[text_col].fillna('').astype(str)
    df[f'{text_col}_len'] = s.str.len()
    df[f'{text_col}_words'] = s.str.split().apply(len)
    df[f'{text_col}_uniq_words'] = s.apply(lambda x: len(set(x.split())))
    df[f'{text_col}_digits'] = s.str.count(r'\d')
    df[f'{text_col}_punc'] = s.str.count(r'[^\w\s]', flags=0)
    df[f'{text_col}_upper'] = s.apply(lambda x: sum(ch.isupper() for ch in x))
    df[f'{text_col}_avgw'] = (df[f'{text_col}_len'] / df[f'{text_col}_words'].replace(0, 1)).astype(float)
    df[f'{text_col}_upr'] = (df[f'{text_col}_upper'] / df[f'{text_col}_len'].replace(0, 1)).astype(float)
    return df

def add_emb_stats(df, emb_col):
    e = df[emb_col].apply(lambda v: np.array(v, dtype=np.float32) if isinstance(v, list) else np.array([], dtype=np.float32))
    df['emb_dim'] = e.apply(lambda v: len(v))
    df['emb_min'] = e.apply(lambda v: float(v.min()) if v.size else 0.0)
    df['emb_max'] = e.apply(lambda v: float(v.max()) if v.size else 0.0)
    df['emb_mean'] = e.apply(lambda v: float(v.mean()) if v.size else 0.0)
    df['emb_std'] = e.apply(lambda v: float(v.std()) if v.size else 0.0)
    df['emb_norm'] = e.apply(lambda v: float(np.linalg.norm(v)) if v.size else 0.0)
    return df

train_df = add_text_feats(train_df, 'text')
test_df = add_text_feats(test_df, 'text')
train_df = add_emb_stats(train_df, 'embedding')
test_df = add_emb_stats(test_df, 'embedding')
print(train_df.shape)
train_df.head()

(53494, 222)


,app_name,full_description,shortDescription,labels_str,labels_list,n_labels,text,embedding,tfidf_0,tfidf_1,...,text_punc,text_upper,text_avgw,text_upr,emb_dim,emb_min,emb_max,emb_mean,emb_std,emb_norm
0,Матрона Московская.,Хочешь знать что будет? Загляни в будущие. Мат...,Узнать будущее. Предсказания . Магия волшебног...,lifestyle,[lifestyle],1,Матрона Московская. Узнать будущее. Предсказан...,"[-0.02546943910419941, -0.02646569162607193, -...",0.0,0.000000,...,18,17,6.819149,0.026521,1536,-0.070459,0.064820,-0.000436,0.025512,1.0
1,"Run and Jump - ""Бесконечный раннер""",Run and Jump это новая увлекательная Аркада - ...,Платформенная Аркада - Раннер с захватывающим ...,action|arcade,"[action, arcade]",2,"Run and Jump - ""Бесконечный раннер"" Платформен...","[-0.03978952392935753, -0.004444682039320469, ...",0.0,0.000000,...,29,33,6.976744,0.036667,1536,-0.072544,0.064759,-0.000229,0.025514,1.0
2,Ghost Maze,"Играя за приведение, собирай необходимые комби...",Игра-головоломка. Попробуй найти выход из лаби...,arcade|puzzle,"[arcade, puzzle]",2,Ghost Maze Игра-головоломка. Попробуй найти вы...,"[-0.04193723574280739, -0.0033695693127810955,...",0.0,0.000000,...,8,6,7.793103,0.026549,1536,-0.067927,0.062633,-0.000385,0.025513,1.0
3,LabTools Mobile: ЛАБОРАТОРНЫЕ ПРИБОРЫ,AR-приложение «LabTools Mobile: Лабораторные п...,AR-приложение которое предназначено для изучен...,education,[education],1,LabTools Mobile: ЛАБОРАТОРНЫЕ ПРИБОРЫ AR-прило...,"[-0.044376254081726074, -0.028724001720547676,...",0.0,0.172609,...,182,131,7.384755,0.032195,1536,-0.068226,0.060918,0.000156,0.025515,1.0
4,Mario Anime Coloring,\nРаскрасьте своих любимых персонажей из mario...,Раскрасьте своих любимых персонажей из mario я...,children|family,"[children, family]",2,Mario Anime Coloring Раскрасьте своих любимых ...,"[-0.03006214275956154, -0.011583167128264904, ...",0.0,0.000000,...,61,37,7.191011,0.028906,1536,-0.069395,0.065026,0.000047,0.025515,1.0


In [9]:
def expand_embedding(df, emb_col, prefix='emb'):
    arr = np.vstack(df[emb_col].apply(lambda v: np.array(v, dtype=np.float32)))
    emb_cols = [f'{prefix}_{i}' for i in range(arr.shape[1])]
    emb_df = pd.DataFrame(arr, columns=emb_cols, index=df.index)
    out = pd.concat([df.drop(columns=[emb_col]), emb_df], axis=1)
    return out, emb_cols

train_df, emb_cols = expand_embedding(train_df, 'embedding')
test_df, emb_cols = expand_embedding(test_df, 'embedding')
print(train_df.shape)
train_df.head()

(53494, 1757)


,app_name,full_description,shortDescription,labels_str,labels_list,n_labels,text,tfidf_0,tfidf_1,tfidf_2,...,emb_1526,emb_1527,emb_1528,emb_1529,emb_1530,emb_1531,emb_1532,emb_1533,emb_1534,emb_1535
0,Матрона Московская.,Хочешь знать что будет? Загляни в будущие. Мат...,Узнать будущее. Предсказания . Магия волшебног...,lifestyle,[lifestyle],1,Матрона Московская. Узнать будущее. Предсказан...,0.0,0.000000,0.000000,...,0.013285,-0.027001,-0.006622,-0.048583,-0.023744,0.014363,0.035482,-0.016020,0.008836,0.055232
1,"Run and Jump - ""Бесконечный раннер""",Run and Jump это новая увлекательная Аркада - ...,Платформенная Аркада - Раннер с захватывающим ...,action|arcade,"[action, arcade]",2,"Run and Jump - ""Бесконечный раннер"" Платформен...",0.0,0.000000,0.671788,...,0.003523,-0.048084,-0.013755,-0.050502,-0.024837,0.019303,0.011996,-0.025982,0.033767,0.064759
2,Ghost Maze,"Играя за приведение, собирай необходимые комби...",Игра-головоломка. Попробуй найти выход из лаби...,arcade|puzzle,"[arcade, puzzle]",2,Ghost Maze Игра-головоломка. Попробуй найти вы...,0.0,0.000000,0.000000,...,0.014284,-0.045735,-0.018553,-0.049476,-0.025340,0.018433,0.014861,-0.024793,0.032536,0.062633
3,LabTools Mobile: ЛАБОРАТОРНЫЕ ПРИБОРЫ,AR-приложение «LabTools Mobile: Лабораторные п...,AR-приложение которое предназначено для изучен...,education,[education],1,LabTools Mobile: ЛАБОРАТОРНЫЕ ПРИБОРЫ AR-прило...,0.0,0.172609,0.000000,...,0.009105,-0.037666,0.009979,-0.053618,-0.005597,0.012486,0.016367,-0.015130,0.011178,0.056622
4,Mario Anime Coloring,\nРаскрасьте своих любимых персонажей из mario...,Раскрасьте своих любимых персонажей из mario я...,children|family,"[children, family]",2,Mario Anime Coloring Раскрасьте своих любимых ...,0.0,0.000000,0.000000,...,0.015005,-0.044766,-0.020036,-0.044337,-0.018741,0.017020,0.012762,-0.016670,0.029289,0.062918


In [10]:
MIN_SAMPLES_PER_CLASS = 50

all_labels_flat = [label for labels in train_df['labels_list'] for label in labels]
label_counts = pd.Series(all_labels_flat).value_counts()
small_classes = set(label_counts[label_counts <= MIN_SAMPLES_PER_CLASS].index)

print(sorted(small_classes))

['ar', 'casino', 'gambling', 'games']


In [11]:
def has_at_least_one_valid_label(labels, small_classes):
    return any(label not in small_classes for label in labels)

old_len = len(train_df)
mask_keep = train_df['labels_list'].apply(lambda x: has_at_least_one_valid_label(x, small_classes))
train_df = train_df[mask_keep].reset_index(drop=True)

print(f'deleted {old_len - len(train_df)} samples')

deleted 57 samples


In [12]:
def remove_small_labels(labels, small_classes):
    return [label for label in labels if label not in small_classes]

train_df['labels_list'] = train_df['labels_list'].apply(
    lambda x: remove_small_labels(x, small_classes)
)

train_df = train_df[train_df['labels_list'].apply(len) > 0].reset_index(drop=True)

In [13]:
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(train_df['labels_list'])
classes = list(mlb.classes_)
len(classes)

41

## Train / Val split

In [14]:
text_cols = []
num_cols = [c for c in train_df.columns if c not in ['app_name', 'full_description', 'shortDescription', 'labels_str', 'n_labels', 'labels_list', 'text']]

X = train_df[num_cols + text_cols].copy()

In [15]:
msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.08, random_state=42)
train_idx, val_idx = next(msss.split(X, Y))

# Xcb_tr, Xcb_va = X_cb.iloc[tr_idx].reset_index(drop=True), X_cb.iloc[va_idx].reset_index(drop=True)
X_train, X_val = X.iloc[train_idx].reset_index(drop=True), X.iloc[val_idx].reset_index(drop=True)
Y_train, Y_val = Y[train_idx], Y[val_idx]
print('Split:', X_train.shape, X_val.shape)

Split: (49130, 1750) (4307, 1750)


In [16]:
X_test = test_df[num_cols + text_cols].copy()

## Boostings

In [17]:
RANDOM_STATE = 52

In [18]:
def hitrate_at_k(y_true, y_proba, k=3):
    hits = 0
    for i in range(len(y_true)):
        true_labels = set(np.where(y_true[i] == 1)[0])
        if not true_labels:
            continue
        top_k_pred = set(np.argsort(y_proba[i])[::-1][:k])
        if len(true_labels & top_k_pred) > 0:
            hits += 1
    return hits / len(y_true)

In [19]:
def save_submission(proba_matrix, mlb, test_app_names, filename):
    # Получаем индексы топ-3 предсказаний
    top3_indices = np.argsort(proba_matrix, axis=1)[:, ::-1][:, :3]

    # Преобразуем индексы в названия классов
    top3_labels = [mlb.classes_[idxs] for idxs in top3_indices]

    # Объединяем названия через "|"
    predictions = ['|'.join(labels) for labels in top3_labels]

    # Создаем DataFrame с правильными именами колонок
    submission = pd.DataFrame({
        'app_name': test_app_names,
        'labels_str': predictions
    })

    # Сохраняем в TSV формате
    submission.to_csv(filename, sep='\t', index=False)
    print(f"💾 Сабмит сохранен: {filename}")

In [20]:
import joblib
import os

MODEL_DIR = "models"
os.makedirs(MODEL_DIR, exist_ok=True)

def save_model(model, filename):
    path = os.path.join(MODEL_DIR, filename)
    joblib.dump(model, path)
    print(f"💾 Модель сохранена: {path}")

def load_model(filename):
    path = os.path.join(MODEL_DIR, filename)
    if os.path.exists(path):
        model = joblib.load(path)
        print(f"📂 Модель загружена: {path}")
        return model
    else:
        print(f"⚠️  Модель не найдена: {path}")
        return None

In [21]:
print("🌲 Обучение XGBoost по классам...")

xgb_val_probas = []
xgb_test_probas = []

for i, label in enumerate(tqdm(classes, desc="XGBoost")):
    model_path = os.path.join(MODEL_DIR, f"xgb_{i}_{label}.joblib")
    
    if os.path.exists(model_path):
        print(f"📂 Загружаем XGBoost: {label}")
        model = joblib.load(model_path)
    else:
        print(f"🆕 Обучаем XGBoost: {label}")
        
        # Вычисляем веса для балансировки
        scale_pos_weight = sum(Y_train[:, i] == 0) / max(sum(Y_train[:, i] == 1), 1)
        
        model = XGBClassifier(
            objective='binary:logistic',
            eval_metric='logloss',
            max_depth=8,
            learning_rate=0.05,
            n_estimators=1200,
            subsample=0.8,
            colsample_bytree=0.8,
            scale_pos_weight=scale_pos_weight,
            random_state=RANDOM_STATE,
            n_jobs=1,
            tree_method='gpu_hist',
            predictor='gpu_predictor',
            use_label_encoder=False,
            verbosity=0
        )
        
        # Обучение с ранней остановкой
        model.fit(
            X_train, Y_train[:, i],
            eval_set=[(X_val, Y_val[:, i])],
            early_stopping_rounds=50,
            verbose=200
        )
        
        # Сохраняем
        joblib.dump(model, model_path)
    
    # Предсказание вероятностей
    lgb_val_proba = model.predict_proba(X_val)[:, 1]
    lgb_test_proba = model.predict_proba(X_test)[:, 1]
    
    xgb_val_probas.append(lgb_val_proba)
    xgb_test_probas.append(lgb_test_proba)

# Конвертируем в матрицы
xgb_val_proba_matrix = np.column_stack(xgb_val_probas)
xgb_test_proba_matrix = np.column_stack(xgb_test_probas)

print("✅ XGBoost: обучение завершено")

🌲 Обучение XGBoost по классам...


XGBoost:   0%|          | 0/41 [00:00<?, ?it/s]

🆕 Обучаем XGBoost: action
[0]	validation_0-logloss:0.65942
[200]	validation_0-logloss:0.12446
[400]	validation_0-logloss:0.10326
[454]	validation_0-logloss:0.10423
🆕 Обучаем XGBoost: adsAndServices
[0]	validation_0-logloss:0.65107
[200]	validation_0-logloss:0.03683
[296]	validation_0-logloss:0.03606
🆕 Обучаем XGBoost: adventure
[0]	validation_0-logloss:0.66033
[200]	validation_0-logloss:0.09881
[361]	validation_0-logloss:0.09133
🆕 Обучаем XGBoost: arcade
[0]	validation_0-logloss:0.66398
[200]	validation_0-logloss:0.19068
[400]	validation_0-logloss:0.16797
[452]	validation_0-logloss:0.16844
🆕 Обучаем XGBoost: board
[0]	validation_0-logloss:0.64863
[200]	validation_0-logloss:0.01271
[264]	validation_0-logloss:0.01313
🆕 Обучаем XGBoost: boardAndCard
[0]	validation_0-logloss:0.65021
[200]	validation_0-logloss:0.03115
[307]	validation_0-logloss:0.03090
🆕 Обучаем XGBoost: books
[0]	validation_0-logloss:0.65123
[200]	validation_0-logloss:0.03508
[349]	validation_0-logloss:0.03294
🆕 Обучаем XG

In [23]:
joblib.dump(xgb_val_proba_matrix, "xgb_val_proba.joblib")
joblib.dump(xgb_test_proba_matrix, "xgb_test_proba.joblib")

['xgb_test_proba.joblib']

In [24]:
hr3_xgb = hitrate_at_k(Y_val, xgb_val_proba_matrix, k=3)
print(f"XGBoost HitRate@3: {hr3_xgb:.5f}")

XGBoost HitRate@3: 0.91177


In [25]:
save_submission(xgb_test_proba_matrix, mlb, test_df['app_name'], "xgb_sub1.tsv")

💾 Сабмит сохранен: xgb_sub1.tsv


In [22]:
import gc
from tqdm import tqdm

print("🐱 Обучение CatBoost по классам...")

PROBA_DIR = "proba_matrices"
os.makedirs(PROBA_DIR, exist_ok=True)

# Списки не нужны — будем сохранять на диск
# cat_val_probas = []
# cat_test_probas = []

for i, label in enumerate(tqdm(classes, desc="CatBoost")):
    model_path = os.path.join(MODEL_DIR, f"cat_{i}_{label}.cbm")
    val_proba_path = os.path.join(PROBA_DIR, f"cat_val_proba_{i}.joblib")
    test_proba_path = os.path.join(PROBA_DIR, f"cat_test_proba_{i}.joblib")

    if os.path.exists(val_proba_path) and os.path.exists(test_proba_path):
        print(f"📂 Загружаем предсказания CatBoost: {label}")
        continue  # Пропускаем, если уже есть предсказания

    if os.path.exists(model_path):
        print(f"📂 Загружаем CatBoost: {label}")
        model = CatBoostClassifier()
        model.load_model(model_path)
    else:
        print(f"🆕 Обучаем CatBoost: {label}")

        model = CatBoostClassifier(
            iterations=1000,              # ↓ Уменьшили
            learning_rate=0.05,
            depth=6,                      # ↓ Уменьшили
            loss_function='Logloss',
            eval_metric='Logloss',
            auto_class_weights='Balanced',
            early_stopping_rounds=50,
            random_seed=RANDOM_STATE,
            task_type='GPU',
            devices='0:0',
            verbose=200,
            thread_count=1                # ↓ Исправлено!
        )

        model.fit(
            X_train, Y_train[:, i],
            eval_set=(X_val, Y_val[:, i]),
            use_best_model=True
        )

        model.save_model(model_path)

    # Предсказание вероятностей
    cat_val_proba = model.predict_proba(X_val)[:, 1]
    cat_test_proba = model.predict_proba(X_test)[:, 1]

    # Сохраняем на диск → не держим в памяти!
    joblib.dump(cat_val_proba, val_proba_path)
    joblib.dump(cat_test_proba, test_proba_path)

    # Явно удаляем модель и вызываем сборщик мусора
    del model
    gc.collect()

print("✅ CatBoost: обучение завершено — все предсказания сохранены на диск.")

🐱 Обучение CatBoost по классам...


CatBoost:   0%|          | 0/41 [00:00<?, ?it/s]

🆕 Обучаем CatBoost: action
0:	learn: 0.6397421	test: 0.6416701	best: 0.6416701 (0)	total: 12.4s	remaining: 3h 25m 48s
bestTest = 0.2971531855
bestIteration = 140
Shrink model to first 141 iterations.


CatBoost:   2%|▏         | 1/41 [00:48<32:31, 48.79s/it]

🆕 Обучаем CatBoost: adsAndServices
0:	learn: 0.6402188	test: 0.6454957	best: 0.6454957 (0)	total: 142ms	remaining: 2m 21s
bestTest = 0.2224996032
bestIteration = 148
Shrink model to first 149 iterations.


...

🆕 Обучаем CatBoost: utilities
0:	learn: 0.6177577	test: 0.6200920	best: 0.6200920 (0)	total: 141ms	remaining: 2m 21s
200:	learn: 0.0695721	test: 0.1509759	best: 0.1485532 (169)	total: 25.6s	remaining: 1m 41s
bestTest = 0.148553199
bestIteration = 169
Shrink model to first 170 iterations.


CatBoost:  98%|█████████▊| 40/41 [25:57<00:38, 38.86s/it]

🆕 Обучаем CatBoost: word
0:	learn: 0.6172700	test: 0.6213791	best: 0.6213791 (0)	total: 135ms	remaining: 2m 15s
bestTest = 0.2137636838
bestIteration = 61
Shrink model to first 62 iterations.


CatBoost: 100%|██████████| 41/41 [26:23<00:00, 38.63s/it]

✅ CatBoost: обучение завершено — все предсказания сохранены на диск.


In [23]:
print("📥 Собираем матрицы вероятностей из файлов...")

cat_val_probas = []
cat_test_probas = []

for i, label in enumerate(classes):
    val_proba_path = os.path.join(PROBA_DIR, f"cat_val_proba_{i}.joblib")
    test_proba_path = os.path.join(PROBA_DIR, f"cat_test_proba_{i}.joblib")

    if os.path.exists(val_proba_path) and os.path.exists(test_proba_path):
        cat_val_probas.append(joblib.load(val_proba_path))
        cat_test_probas.append(joblib.load(test_proba_path))
    else:
        raise FileNotFoundError(f"Не найдены предсказания для класса {label}")

cat_val_proba_matrix = np.column_stack(cat_val_probas)
cat_test_proba_matrix = np.column_stack(cat_test_probas)

print(f"📊 Размер матриц: val={cat_val_proba_matrix.shape}, test={cat_test_proba_matrix.shape}")

📥 Собираем матрицы вероятностей из файлов...
📊 Размер матриц: val=(4307, 41), test=(15046, 41)


In [25]:
hr3_cb = hitrate_at_k(Y_val, cat_val_proba_matrix, k=3)
print(f"Catboost HitRate@3: {hr3_cb:.5f}")

Catboost HitRate@3: 0.88507


In [26]:
save_submission(cat_test_proba_matrix, mlb, test_df['app_name'], "cb_sub1.tsv")

💾 Сабмит сохранен: cb_sub1.tsv


In [28]:
joblib.dump(cat_val_proba_matrix, "cb_val_proba.joblib")
joblib.dump(cat_test_proba_matrix, "cb_test_proba.joblib")

['cb_test_proba.joblib']

In [ ]:
lgb_val_probas = []
lgb_test_probas = []

for i, label in enumerate(tqdm(classes, desc="LightGBM")):
    model_path = os.path.join(MODEL_DIR, f"lgb_{i}_{label}.joblib")

    if os.path.exists(model_path):
        print(f"📂 Загружаем LightGBM: {label}")
        model = joblib.load(model_path)
    else:
        print(f"🆕 Обучаем LightGBM: {label}")

        # Создаём датасеты
        train_data = lgb.Dataset(X_train, label=Y_train[:, i])
        val_data = lgb.Dataset(X_val, label=Y_val[:, i], reference=train_data)

        # Параметры
        params = {
            'objective': 'binary',
            'boosting_type': 'gbdt',
            'num_leaves': 63,
            'learning_rate': 0.03,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'is_unbalance': True,
            'random_state': RANDOM_STATE,
            'verbosity': -1,
            'device': 'gpu',
            'gpu_platform_id': 0,
            'gpu_device_id': 0
        }

        # Callbacks
        callbacks = [
            lgb.early_stopping(stopping_rounds=50, verbose=False),
            lgb.log_evaluation(period=200)
        ]

        # Обучение
        model = lgb.train(
            params,
            train_data,
            num_boost_round=1200,
            valid_sets=[val_data],
            callbacks=callbacks
        )

        # Сохраняем
        joblib.dump(model, model_path)

    # Предсказание
    lgb_val_proba = model.predict(X_val)
    lgb_test_proba = model.predict(X_test)

    lgb_val_probas.append(lgb_val_proba)
    lgb_test_probas.append(lgb_test_proba)

# Конвертируем в матрицы
lgb_val_proba_matrix = np.column_stack(lgb_val_probas)
lgb_test_proba_matrix = np.column_stack(lgb_test_probas)

LightGBM:   0%|          | 0/41 [00:00<?, ?it/s]

🆕 Обучаем LightGBM: action
🆕 Обучаем LightGBM: adsAndServices
[200]	valid_0's binary_logloss: 0.0410955
🆕 Обучаем LightGBM: adventure
🆕 Обучаем LightGBM: arcade
🆕 Обучаем LightGBM: board
🆕 Обучаем LightGBM: boardAndCard
[200]	valid_0's binary_logloss: 0.0336743
🆕 Обучаем LightGBM: books
[200]	valid_0's binary_logloss: 0.038053
🆕 Обучаем LightGBM: business
[200]	valid_0's binary_logloss: 0.107025
[400]	valid_0's binary_logloss: 0.0969849
🆕 Обучаем LightGBM: card
🆕 Обучаем LightGBM: casual
🆕 Обучаем LightGBM: children
[200]	valid_0's binary_logloss: 0.0841547
🆕 Обучаем LightGBM: education
[200]	valid_0's binary_logloss: 0.0766164
[400]	valid_0's binary_logloss: 0.0725932
🆕 Обучаем LightGBM: entertainment
🆕 Обучаем LightGBM: family
[200]	valid_0's binary_logloss: 0.0296803
🆕 Обучаем LightGBM: finance
🆕 Обучаем LightGBM: foodAndDrink
🆕 Обучаем LightGBM: health
🆕 Обучаем LightGBM: indie
[200]	valid_0's binary_logloss: 0.0487946
🆕 Обучаем LightGBM: lifestyle
🆕 Обучаем LightGBM: music
🆕 Обуча

In [132]:
# base_lgb = LGBMClassifier(
#     objective='binary',
#     boosting_type='gbdt',
#     num_leaves=63,
#     learning_rate=0.03,
#     n_estimators=1200,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     reg_alpha=0.1,
#     reg_lambda=0.1,
#     is_unbalance=True,
#     random_state=RANDOM_STATE,
#     n_jobs=1,
#     device='gpu',                    # ← Включение GPU
#     gpu_platform_id=0,               # ← платформа (обычно 0)
#     gpu_device_id=0,                 # ← устройство (обычно 0)
#     verbose=200
# )

# lgb_multi = MultiOutputClassifier(base_lgb, n_jobs=-1)
# lgb_multi.fit(X_train, Y_train)
# save_model(lgb_multi, "lgb_multi.joblib")

ValueError: For early stopping, at least one dataset and eval metric is required for evaluation

In [76]:
hr3_lgb = hitrate_at_k(Y_val, lgb_val_proba_matrix, k=3)
print(f"LightGBM HitRate@3: {hr3_lgb:.5f}")

LightGBM HitRate@3: 0.90945


In [93]:
save_submission(lgb_test_proba_matrix, mlb, test_df['app_name'], "lgb_sub.tsv")

💾 Сабмит сохранен: lgb_sub.tsv


In [ ]:
joblib.dump(lgb_val_proba_matrix, os.path.join(PROBA_DIR, "lgb_val_proba.joblib"))
joblib.dump(lgb_test_proba_matrix, os.path.join(PROBA_DIR, "lgb_test_proba.joblib"))